In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

In [ ]:
test=pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

In [ ]:
fig=plt.figure()
sns.countplot(train.loc[train.Sex=='female']['Survived'],data=train)
female=train.loc[train.Sex=='female']['Survived']
rate=sum(female)/len(female)
print("Rate of women surviving is",rate)

In [ ]:
fig=plt.figure()
sns.countplot(train.loc[train.Sex=='male']['Survived'],data=train)
male = train.loc[train.Sex == 'male']["Survived"]
rate_m = sum(male)/len(male)

print("% of men who survived:", rate_m)

In [ ]:
sns.barplot(x="Sex",y="Survived",data=train)

In [ ]:
sns.barplot(x="Pclass", y="Survived", data=train)

In [ ]:
g  = sns.factorplot(x="Parch",y="Survived",data=train, size = 8)
g = g.set_ylabels("Survival Percentage")

In [ ]:
g  = sns.factorplot(x="SibSp",y="Survived",data=train, size = 8)
g = g.set_ylabels("Survival Percentage")

In [ ]:
dataset = pd.concat([train, test], sort=False, ignore_index=True)
dataset.head()

In [ ]:
dataset['FamSize'] = dataset['SibSp'] + dataset['Parch'] + 1
g  = sns.factorplot(x="FamSize",y="Survived",data=dataset, size = 8)
g = g.set_ylabels("Survival Percentage")

In [ ]:
dataset['Single'] = dataset['FamSize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['FamSize'].map(lambda s: 1 if  s == 2  else 0)
dataset['MedF'] = dataset['FamSize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['FamSize'].map(lambda s: 1 if s >= 5 else 0)

In [ ]:
g = sns.factorplot(x="Single",y="Survived",data=dataset,kind="bar")
g = g.set_ylabels("Survival Probability")
g = sns.factorplot(x="SmallF",y="Survived",data=dataset,kind="bar")
g = g.set_ylabels("Survival Probability")
g = sns.factorplot(x="MedF",y="Survived",data=dataset,kind="bar")
g = g.set_ylabels("Survival Probability")
g = sns.factorplot(x="LargeF",y="Survived",data=dataset,kind="bar")
g = g.set_ylabels("Survival Probability")

In [ ]:
dataset.isnull().mean().sort_values(ascending=False)

In [ ]:
dataset.drop(['Cabin','Ticket','Name'],axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset = pd.get_dummies(dataset, columns = ["Embarked"])
dataset.drop(['SibSp','Parch'],axis=1,inplace=True)
dataset.head()

In [ ]:
s_map={'male':0,'female':1}
dataset['Sex']=dataset['Sex'].map(s_map)
dataset.head()

In [ ]:
dataset['Age'] = dataset.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
dataset.head()

In [ ]:
dataset.isnull().mean().sort_values(ascending=False)

In [ ]:
dataset['Embarked'] = dataset['Embarked'].fillna('S')
med_fare = dataset.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]

dataset['Fare'] = dataset['Fare'].fillna(med_fare)
dataset.head()

In [ ]:
dataset.isnull().mean().sort_values(ascending=False)

In [ ]:
train=dataset[:len(train)]

test=dataset[len(train):]

test.drop(labels=['Survived'], axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['Survived']=train['Survived'].astype('int')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train["Survived"]

features = ["Pclass", "Sex","Fare","FamSize","Single","SmallF","MedF","LargeF","Embarked_C","Embarked_Q","Embarked_S"]
X = train[features]
X_test = test[features]

model = RandomForestClassifier(n_estimators=10, max_depth=4, random_state=3,criterion='entropy')
model.fit(X, y)
predictions = model.predict(X_test)

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
parameters=[{'n_estimators':[10,100,700],'criterion':['entropy','gini'],'max_depth':[2,3,4,5],'random_state':[1,2,3],
             'min_samples_leaf':[1,2,3]}]
grid_search=GridSearchCV(estimator=model,
                         param_grid=parameters,
                        scoring='accuracy',
                        cv=10,
                        n_jobs=-1)
grid_search=grid_search.fit(X,y)
'''

In [ ]:
#best=grid_search.best_score_
#best

In [ ]:
#params_b=grid_search.best_params_
#params_b

In [ ]:
'''
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = X, y = y, cv = 10)
'''

In [ ]:
#accuracies

In [ ]:
#print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
#print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")